In [ ]:
%tensorflow_version 1.x #use tensorflow magic to use version 1.x in colab

TensorFlow 1.x selected.


In [ ]:
#imports
import pandas as pd
import numpy as np
import nltk
import io
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from keras.preprocessing import text as keras_text, sequence as keras_seq
from keras.utils import np_utils
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers import Dense, Activation, Convolution1D, MaxPooling1D, Dropout, Flatten
from keras.models import Sequential, save_model
from sklearn.model_selection import train_test_split
from keras.layers import Embedding
from sklearn.metrics import classification_report,confusion_matrix

Using TensorFlow backend.


In [ ]:
#traning data
df_train=pd.read_csv("/content/drive/My Drive/minor/english_dataset/english_dataset_prepro.csv")

In [ ]:
df_train.head()

,text_id,text,task_1,task_2,task_3
0,hasoc_en_1,dhonikeepstheglove watch sports minister kiren...,NOT,NONE,NONE
1,hasoc_en_2,politico remember clearly individual1 admitted...,HOF,HATE,TIN
2,hasoc_en_3,cricketworldcup guess would winner cwc19 team ...,NOT,NONE,NONE
3,hasoc_en_4,corbyn politically intellectual borisjohnsonsh...,NOT,NONE,NONE
4,hasoc_en_5,best teamindia another swimming competition su...,NOT,NONE,NONE


In [ ]:
Column_Sequence=np.array(df_train["text"],dtype="str") #storing value  of tweets or text in numpy array of strings

In [ ]:
Tokenizer=RegexpTokenizer(r"\w+[a-z]+") #create a reg exp tokenizer object for text to make a list of strings and lowercase text (earlier also removed stopwords but that has been moved to preprocessing code)

In [ ]:
fresh=[]
for ix in Column_Sequence:
    sen=ix.lower()                        #lower case
    sen=Tokenizer.tokenize(sen)           #tokenize the sentences.
    wordss=[(Word) for Word in sen]
    sentance=" ".join(wordss)
    fresh.append(sentance)                #append all in one.

In [ ]:
tokenizer = keras_text.Tokenizer(char_level = False,lower=True) #create a kerastokenizer object
t=tokenizer.fit_on_texts(list(fresh))                           #fit on training text
list_tokenized_train = tokenizer.texts_to_sequences(fresh)
X_t = keras_seq.pad_sequences(list_tokenized_train, maxlen=300,padding="post") #maxlen after padding 300
word_index = tokenizer.word_index      #get a list of all tokens or words in the vocab

In [ ]:
#extract word2vec vectors for words in vocab
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors
#load word2vec vectors from file.
word_vectors = KeyedVectors.load_word2vec_format('/content/drive/My Drive/minor/wordtovec/GoogleNews-vectors-negative300.bin.gz', binary=True)

EMBEDDING_DIM=300
vocabulary_size=len(word_index)+1
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_index.items():
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector            #store the vector if found.
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)  #store a rndom vector if not found

del(word_vectors)  #del word2vec vectors to free memory

from keras.layers import Embedding
#create an embediing layer from word2vec vectors of our vocab
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
len(tokenizer.word_counts)

16526

In [ ]:
X_train=X_t       #training text

In [ ]:
X_train.shape

(5852, 300)

In [ ]:
Y=df_train["task_1"]    #labels of classes with strings.

In [ ]:
Y = Y.map({'HOF':0, 'NOT': 1})   #map the class names to numbers.

In [ ]:
# One-hot encode label
Y_train=np_utils.to_categorical(Y)

In [ ]:
Y_train.shape

(5852, 2)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_train,Y_train, test_size=0.10, random_state=42) #training validation split

In [ ]:
y_train.shape

(5266, 2)

In [ ]:
x_train.shape

(5266, 300)

In [ ]:
max_len=300   #length of text sequence

In [ ]:
x_train

array([[    4,    21,   188, ...,     0,     0,     0],
       [ 1495,  1247,    70, ...,     0,     0,     0],
       [ 9400,  9401,  2422, ...,     0,     0,     0],
       ...,
       [ 3805,  2826, 15341, ...,     0,     0,     0],
       [  665, 15679,   968, ...,     0,     0,     0],
       [  382,  3152,  8291, ...,     0,     0,     0]], dtype=int32)

In [ ]:
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
#CNN
sequence_input1 = Input(shape=(max_len,), dtype='int32')  #input layer
embedded_sequences1 = embedding_layer(sequence_input1)
l_cov1= Conv1D(128, 5, activation='relu')(embedded_sequences1)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(5)(l_cov2)
l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(5)(l_cov3)  # global max pooling
l_flat1 = Flatten()(l_pool3)
l_dense1 = Dense(128, activation='relu')(l_flat1)
preds1 = Dense(2, activation='softmax')(l_dense1)        #output layer

model1 = Model(sequence_input1, preds1)
model1.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("Simplified convolutional neural network")
model1.summary()
cp1=ModelCheckpoint('model_cnn.hdf5',monitor='val_acc',verbose=1,save_best_only=True)  #callback checkpoint


Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Simplified convolutional neural network
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 300, 300)          4958100   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 296, 128)          192128    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 59, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 55, 128)           82048     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 11, 128)           0         
___

In [ ]:
from keras.layers import Dense, Input, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, CuDNNGRU, Bidirectional, concatenate
from keras.models import Model
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
#BiGRU
sequence_input2 = Input(shape=(max_len,), dtype='int32') #input layer
embedded_sequences2 = embedding_layer(sequence_input2)
gru1=Bidirectional(CuDNNGRU(128, return_sequences=True))(embedded_sequences2)  #gpu optimized gru layer with a bi-directional layer
gru2= Bidirectional(CuDNNGRU(128, return_sequences=True))(gru1)
avg_pool = GlobalAveragePooling1D()(gru2)
max_pool = GlobalMaxPooling1D()(gru2)
conc = concatenate([avg_pool, max_pool])
l_dense2 = Dense(128, activation='relu')(conc)
preds2 = Dense(2, activation='softmax')(l_dense2)  #output layer

model2 = Model(sequence_input2, preds2)
model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

print("GRU")
model2.summary()
cp2=ModelCheckpoint('model_gru.hdf5',monitor='val_accuracy',verbose=1,save_best_only=True) #callback checkpoint

GRU
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 300)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 300, 300)     4958100     input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 300, 256)     330240      embedding_1[1][0]                
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 300, 256)     296448      bidirectional_1[0][0]            
________________________________________________________________________________________

In [ ]:
from keras.layers import Dense, Input, GlobalMaxPooling1D, CuDNNLSTM, GlobalAveragePooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM, Bidirectional, concatenate
from keras.models import Model
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
#Bi-LSTM
sequence_input3 = Input(shape=(max_len,), dtype='int32') #input layer
embedded_sequences3 = embedding_layer(sequence_input3)
lstm1 = Bidirectional(CuDNNLSTM(128, return_sequences=True))(embedded_sequences3) #gpu optimized lstm layer with a bi-directional layer
lstm2= Bidirectional(CuDNNLSTM(128, return_sequences=True))(lstm1)
avg_pool = GlobalAveragePooling1D()(lstm2)
max_pool = GlobalMaxPooling1D()(lstm2)
conc = concatenate([avg_pool, max_pool])
l_dense3 = Dense(128, activation='relu')(conc)
preds3 = Dense(2, activation='softmax')(l_dense3) #output layer

model3 = Model(sequence_input3, preds3)
model3.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

print("LSTM")
model3.summary()
cp3=ModelCheckpoint('model_lstm.hdf5',monitor='val_accuracy',verbose=1,save_best_only=True) #callback checkpoint

LSTM
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 300)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 300, 300)     4958100     input_3[0][0]                    
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 300, 256)     440320      embedding_1[2][0]                
__________________________________________________________________________________________________
bidirectional_4 (Bidirectional) (None, 300, 256)     395264      bidirectional_3[0][0]            
_______________________________________________________________________________________

In [ ]:
history1=model1.fit(x_train, y_train, validation_data=(x_test, y_test),epochs=15, batch_size=32,callbacks=[cp1]) #training cnn


Train on 5266 samples, validate on 586 samples
Epoch 1/15
5266/5266 [==============================] - 3s 527us/step - loss: 0.6514 - acc: 0.6231 - val_loss: 0.6330 - val_acc: 0.6348

Epoch 00001: val_acc improved from -inf to 0.63481, saving model to model_cnn.hdf5
Epoch 2/15
5266/5266 [==============================] - 1s 194us/step - loss: 0.5748 - acc: 0.7070 - val_loss: 0.6779 - val_acc: 0.6007

Epoch 00002: val_acc did not improve from 0.63481
Epoch 3/15
5266/5266 [==============================] - 1s 188us/step - loss: 0.4188 - acc: 0.8118 - val_loss: 0.8247 - val_acc: 0.6195

Epoch 00003: val_acc did not improve from 0.63481
Epoch 4/15
5266/5266 [==============================] - 1s 191us/step - loss: 0.3007 - acc: 0.8781 - val_loss: 0.8343 - val_acc: 0.6468

Epoch 00004: val_acc improved from 0.63481 to 0.64676, saving model to model_cnn.hdf5
Epoch 5/15
5266/5266 [==============================] - 1s 191us/step - loss: 0.2130 - acc: 0.9201 - val_loss: 1.3484 - val_acc: 0.6382

In [ ]:
history2=model2.fit(x_train, y_train, validation_data=(x_test, y_test),epochs=7, batch_size=32,callbacks=[cp2]) #training bi-gru

Train on 5266 samples, validate on 586 samples
Epoch 1/7
5266/5266 [==============================] - 10s 2ms/step - loss: 0.6432 - acc: 0.6310 - val_loss: 0.6036 - val_acc: 0.6792

Epoch 00001: val_acc improved from -inf to 0.67918, saving model to model_gru.hdf5
Epoch 2/7
5266/5266 [==============================] - 10s 2ms/step - loss: 0.5846 - acc: 0.6897 - val_loss: 0.5958 - val_acc: 0.6962

Epoch 00002: val_acc improved from 0.67918 to 0.69625, saving model to model_gru.hdf5
Epoch 3/7
5266/5266 [==============================] - 10s 2ms/step - loss: 0.5177 - acc: 0.7467 - val_loss: 0.6101 - val_acc: 0.6792

Epoch 00003: val_acc did not improve from 0.69625
Epoch 4/7
5266/5266 [==============================] - 10s 2ms/step - loss: 0.4125 - acc: 0.8078 - val_loss: 0.6585 - val_acc: 0.6638

Epoch 00004: val_acc did not improve from 0.69625
Epoch 5/7
5266/5266 [==============================] - 10s 2ms/step - loss: 0.2812 - acc: 0.8863 - val_loss: 0.7656 - val_acc: 0.6706

Epoch 000

In [ ]:
history3=model3.fit(x_train, y_train, validation_data=(x_test, y_test),epochs=7, batch_size=32,callbacks=[cp3]) #training bilstm

Train on 5266 samples, validate on 586 samples
Epoch 1/7
5266/5266 [==============================] - 12s 2ms/step - loss: 0.6371 - acc: 0.6329 - val_loss: 0.6303 - val_acc: 0.6212

Epoch 00001: val_acc improved from -inf to 0.62116, saving model to model_lstm.hdf5
Epoch 2/7
5266/5266 [==============================] - 11s 2ms/step - loss: 0.5948 - acc: 0.6895 - val_loss: 0.5803 - val_acc: 0.6980

Epoch 00002: val_acc improved from 0.62116 to 0.69795, saving model to model_lstm.hdf5
Epoch 3/7
5266/5266 [==============================] - 11s 2ms/step - loss: 0.5481 - acc: 0.7235 - val_loss: 0.5996 - val_acc: 0.6945

Epoch 00003: val_acc did not improve from 0.69795
Epoch 4/7
5266/5266 [==============================] - 11s 2ms/step - loss: 0.4790 - acc: 0.7689 - val_loss: 0.6444 - val_acc: 0.6638

Epoch 00004: val_acc did not improve from 0.69795
Epoch 5/7
5266/5266 [==============================] - 11s 2ms/step - loss: 0.3821 - acc: 0.8314 - val_loss: 0.6927 - val_acc: 0.6706

Epoch 0

In [ ]:
w1=model1.predict(x_test) #predict on val data cnn
w1=np.argmax(w1,axis=1)
w2=model2.predict(x_test) #predict on val data bi-gru
w2=np.argmax(w2,axis=1)
w3=model3.predict(x_test) #predict on val data bi-lstm
w3=np.argmax(w3,axis=1)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
#true labels for val data
Y_actual=[]
for ix in y_test:
    Y_actual.append(np.argmax(ix))

In [ ]:
print("CNN")
print(classification_report(Y_actual,w1))
print("BI-GRU")
print(classification_report(Y_actual,w2))
print("BI-LSTM")
print(classification_report(Y_actual,w3))

CNN
              precision    recall  f1-score   support

           0       0.56      0.57      0.56       230
           1       0.72      0.72      0.72       356

    accuracy                           0.66       586
   macro avg       0.64      0.64      0.64       586
weighted avg       0.66      0.66      0.66       586

BI-GRU
              precision    recall  f1-score   support

           0       0.58      0.53      0.55       230
           1       0.71      0.76      0.73       356

    accuracy                           0.67       586
   macro avg       0.65      0.64      0.64       586
weighted avg       0.66      0.67      0.66       586

BI-LSTM
              precision    recall  f1-score   support

           0       0.59      0.46      0.52       230
           1       0.69      0.79      0.74       356

    accuracy                           0.66       586
   macro avg       0.64      0.63      0.63       586
weighted avg       0.65      0.66      0.65       586



In [ ]:
data_test=pd.read_csv('/content/drive/My Drive/minor/english_dataset/hasoc2019_en_test-2919_prepro.csv') #testing data

In [ ]:
data_test.head()

,text_id,text,task_1,task_2,task_3
0,hasoc_en_902,west bengal doctor crisis protesting doctors a...,NOT,NONE,NONE
1,hasoc_en_416,68 5 million people forced leave homes read,NOT,NONE,NONE
2,hasoc_en_207,came saw look fort good luck,NOT,NONE,NONE
3,hasoc_en_595,get brexit delivered october 31st help build m...,NOT,NONE,NONE
4,hasoc_en_568,fuck go back dark ages cow ibnliverealtime rap...,HOF,PRFN,UNT


In [ ]:
data_test.text=data_test.text.astype(str)
df_text=data_test['text']

In [ ]:
Column_Sequence=np.array(data_test["text"],dtype="str")
fresh=[]
for ix in Column_Sequence:
    sen=ix.lower()
    sen=Tokenizer.tokenize(sen)    #use the reg exp tokenizer object for text to make a list of strings and lowercase text (earlier also removed stopwords but that has been moved to preprocessing code)
    wordss=[(Word) for Word in sen]
    sentance=" ".join(wordss)
    fresh.append(sentance)

In [ ]:
tokenizer = keras_text.Tokenizer(char_level = False,lower=True) #create a kerastokenizer object
t=tokenizer.fit_on_texts(list(fresh))
list_tokenized_train = tokenizer.texts_to_sequences(fresh)
X_test = keras_seq.pad_sequences(list_tokenized_train, maxlen=300,padding="post") #maxlen after padding 300

In [ ]:
w_test1=model1.predict(X_test)     #predict class of testing data for cnn
w_test1=np.argmax(w_test1,axis=1)
w_test2=model2.predict(X_test)     #predict class of testing data for bi-gru
w_test2=np.argmax(w_test2,axis=1)
w_test3=model3.predict(X_test)     #predict class of testing data for bi-lstm
w_test3=np.argmax(w_test3,axis=1)

In [ ]:
data_test['task_1'] = data_test['task_1'].map({'HOF':0, 'NOT': 1}) #map label strings to numbers

In [ ]:
y_test_actual=data_test['task_1']

In [ ]:
print("CNN")
print(classification_report(y_test_actual,w_test1))
print("BI-GRU")
print(classification_report(y_test_actual,w_test2))
print("BI-LSTM")
print(classification_report(y_test_actual,w_test3))

CNN
              precision    recall  f1-score   support

           0       0.22      0.32      0.26       288
           1       0.73      0.63      0.68       865

    accuracy                           0.55      1153
   macro avg       0.48      0.47      0.47      1153
weighted avg       0.61      0.55      0.57      1153

BI-GRU
              precision    recall  f1-score   support

           0       0.22      0.31      0.26       288
           1       0.74      0.65      0.69       865

    accuracy                           0.56      1153
   macro avg       0.48      0.48      0.47      1153
weighted avg       0.61      0.56      0.58      1153

BI-LSTM
              precision    recall  f1-score   support

           0       0.27      0.31      0.29       288
           1       0.76      0.72      0.74       865

    accuracy                           0.62      1153
   macro avg       0.51      0.51      0.51      1153
weighted avg       0.64      0.62      0.63      1153

